In [232]:
import pandas as pd
import numpy as np 
from itertools import permutations
import math
import random

In [233]:
def distance_matrix(pattern):
    # to get the side length, solve for n where len(pattern) = n*(n + 1)/2 (triangular number formula)
    side_length = (int(math.sqrt(1 + 8 * len(pattern))) - 1) / 2 + 1
    side_length = int(side_length)
    assert (side_length * (side_length - 1)) // 2 == len(pattern), "Pattern length must be a triangular number."
    # create the grid
    grid = [[0] *(side_length) for i in range((side_length))]

    # fill in the grid
    position = 0
    for i in range(0, side_length - 1):
        for j in range(0, side_length - 1 - i):
            element = pattern[position]; position += 1
            grid[i][i + j + 1] = element # fill in the upper triangle
            grid[i + j + 1][i] = element # fill in the lower triangle
    return grid

def matrix_to_string(matrix): 
    return "\n".join("\t".join(str(x) for x in row) for row in distance_matrix(num))
                     #([1, 2, 4, 3, 5, 6]))


In [234]:
#generate random numbers
i = random.randint(1,101)
print(i)
output = []
for j in range(i,i+6,1):
    output.append(j)
output
data = distance_matrix(output)
data

95


[[0, 95, 96, 97], [95, 0, 98, 99], [96, 98, 0, 100], [97, 99, 100, 0]]

In [235]:
df = pd.DataFrame(data, columns=['A', 'B', 'C', 'D'], index = labels)
#convert to a dictionary 
data_dict = {}
for idx, col in df.iterrows():
    data_dict[idx] = col.values
    
dm = pd.DataFrame.from_dict(data_dict)
dm = mat.rename(index={0: 'A', 1: 'B', 2: 'C', 3:'D'})
dm

,A,B,C,D
A,0,61,62,63
B,61,0,64,65
C,62,64,0,66
D,63,65,66,0


In [236]:
#check if the matrix is additive using four point condition
def is_additive(matrix, labels):
    i = labels[0]
    j = labels[1] 
    k = labels[2] 
    l = labels[3] 
    result = ''
    if ((matrix[i][j] + matrix[k][l] <= matrix[i][k] + matrix[j][l]) and 
    (matrix[i][k] + matrix[j][l] == matrix[i][l] + matrix[j][k])):
        result = "Additive"
    else:
        result = "Not additive"
    return result 

In [237]:
def is_additive_full(dm):
    cols = 'ABCD'
    labels = permutations(cols,4)
    check = []
    for combs in labels:
        new_label = []
        for item in combs:
            new_label+=item
        check.append(is_additive(dm, new_label))
    return (set(check))

is_additive_full(dm)

{'Additive'}

## Neigbor join

In [238]:
import copy
clusters = []
for cluster in list(dm):
    clusters.append(cluster)
cluster2idx = { name : idx for idx, name in enumerate(clusters) }
idx2cluster = copy.deepcopy(clusters)
m = copy.deepcopy(dm).to_dict()
Z = np.empty((0,4), float)
result = ""
print(clusters)
#membership = [set([cluster2idx[name]]) for name in clusters]

['A', 'B', 'C', 'D']


In [239]:
M, N = dm.shape
new_cluster_idx = M+1
while len(clusters) > 1:
    print(result)
    print(m)
    idx1 = None
    idx2 = None
    ui = 0
    uj = 0
    Sd = 100000
    for i in clusters:
        for j in clusters:
            if i != j:
                ui = sum(m[i][k] for k in clusters)
                uj = sum(m[j][k] for k in clusters)
                curr = (M - 2) * m[i][j] - ui - uj
                if Sd > curr:
                    Sd = curr
                    idx1 = cluster2idx[i]
                    idx2 = cluster2idx[j]
    
    Dij = m[idx2cluster[idx1]][idx2cluster[idx2]]
    new_cluster = "(" + idx2cluster[idx1] +"." + idx2cluster[idx2] + ")"
    result = new_cluster
    m[new_cluster] = {}
    new_cluster_idx += 1
    #membership.append(membership[cluster2idx[idx1]] | membership[cluster2idx[idx2]])
    #Z = np.append(Z, np.array([[idx1, idx2, Sd]]))
    
    m[idx2cluster[idx1]][new_cluster] = (m[idx2cluster[idx1]][idx2cluster[idx2]] + (1/(M-2)) * (ui - uj))/2
    m[idx2cluster[idx2]][new_cluster] = (m[idx2cluster[idx1]][idx2cluster[idx2]] + (1/(M-2)) * (uj - ui))/2
    
    #if idx2cluster[idx1] in m: del m[idx2cluster[idx1]]
    #if idx2cluster[idx2] in m: del m[idx2cluster[idx2]]
    
    clusters.remove(idx2cluster[idx1])
    clusters.remove(idx2cluster[idx2])
    
    m[new_cluster][new_cluster] = 0
    for c in clusters:
        m[c][new_cluster] = (m[c][idx2cluster[idx1]] + m[c][idx2cluster[idx2]] - Dij)/2
        m[new_cluster][c] = m[c][new_cluster]
        for k in list(m[c].keys()):
            if k == idx2cluster[idx1] or k == idx2cluster[idx2]:
                del m[c][k]
        
    del cluster2idx[idx2cluster[idx1]]
    del cluster2idx[idx2cluster[idx2]]
    if idx2cluster[idx1] in m: del m[idx2cluster[idx1]]
    if idx2cluster[idx2] in m: del m[idx2cluster[idx2]]
    
    clusters.append(new_cluster)
    cluster2idx[new_cluster] = len(idx2cluster)
    idx2cluster.append(new_cluster)
        
    #M -= 1
    #N -= 1


{'A': {'A': 0, 'B': 61, 'C': 62, 'D': 63}, 'B': {'A': 61, 'B': 0, 'C': 64, 'D': 65}, 'C': {'A': 62, 'B': 64, 'C': 0, 'D': 66}, 'D': {'A': 63, 'B': 65, 'C': 66, 'D': 0}}
(A.B)
{'C': {'C': 0, 'D': 66, '(A.B)': 32.5}, 'D': {'C': 66, 'D': 0, '(A.B)': 33.5}, '(A.B)': {'(A.B)': 0, 'C': 32.5, 'D': 33.5}}
(C.(A.B))
{'D': {'D': 0, '(C.(A.B))': 33.5}, '(C.(A.B))': {'(C.(A.B))': 0, 'D': 33.5}}


In [240]:
print(result)

(D.(C.(A.B)))


In [241]:
##additive phlyogeny construction using degenerate triples

In [242]:
def additive_phylogeny(D,T):
    if D.shape == (2,2):
        T = {dm.index[0] : (dm.index[1], dm[dm.index[0]][dm.index[1]])}
        return T
    i,j,k,flag = degenerate_check(D)
    if



SyntaxError: invalid syntax (<ipython-input-242-1bbd5d22e6ed>, line 7)